In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

In [2]:
df = pd.read_csv("GymAndDietRecommendationCleaned.csv")

X = df[['Sex', 'Hypertension',  'Level', 'Age', 'Height', 'Exercises_Label']]
y = df['Diet_Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [3]:
class_weights = dict(zip(np.unique(y_train), len(y_train) / (len(np.unique(y_train)) * np.bincount(y_train))))
class_weights = {i: class_weights.get(i, 1.0) for i in range(len(np.unique(y)))}

xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb.fit(X_train, y_train, sample_weight=[class_weights.get(y, 1.0) for y in y_train])
xgb_pred = xgb.predict(X_test)

C:\ProgramData\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:09:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [4]:
print("XGBoost Results for Diet_Label:")
print("Accuracy:", accuracy_score(y_test, xgb_pred))
print("\nClassification Report:\n", classification_report(y_test, xgb_pred, zero_division=0))

xgb_importance = pd.DataFrame({'Feature': X.columns, 'Importance': xgb.feature_importances_}).sort_values(by='Importance', ascending=False)
print("\nXGBoost Feature Importance:\n", xgb_importance)
print("\nFeature Correlation:\n" , X.corr())

XGBoost Results for Diet_Label:
Accuracy: 0.9406919275123559

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.97      0.97       405
           1       0.74      0.69      0.71        80
           2       0.66      0.74      0.70        80
           3       0.24      0.80      0.36         5
           4       0.99      0.95      0.97       831
           5       0.95      0.98      0.96       141
           6       0.96      0.95      0.96       280
           7       0.91      0.96      0.93       139
           8       0.94      0.96      0.95       139
           9       0.96      0.97      0.97       188
          10       0.93      0.94      0.94       140

    accuracy                           0.94      2428
   macro avg       0.84      0.90      0.86      2428
weighted avg       0.95      0.94      0.94      2428


XGBoost Feature Importance:
            Feature  Importance
5  Exercises_Label    0.393123
1     Hyp